In [84]:
from __future__ import print_function, division
import SimpleITK as sitk
import numpy as np
import os
from glob import glob
import pandas as pd

tqdm = lambda x: x

In [85]:
def load_itkfilewithtrucation(filename, upper=200, lower=-200):
    # 1,tructed outside of liver value
    srcitkimage = sitk.Cast(sitk.ReadImage(filename), sitk.sitkFloat32)
    srcitkimagearray = sitk.GetArrayFromImage(srcitkimage)
    srcitkimagearray[srcitkimagearray > upper] = upper
    srcitkimagearray[srcitkimagearray < lower] = lower
    # 2,get tructed outside of liver value image
    sitktructedimage = sitk.GetImageFromArray(srcitkimagearray)
    origin = np.array(srcitkimage.GetOrigin())
    spacing = np.array(srcitkimage.GetSpacing())
    sitktructedimage.SetSpacing(spacing)
    sitktructedimage.SetOrigin(origin)
    # 3 normalization value to 0-255
    rescalFilt = sitk.RescaleIntensityImageFilter()
    rescalFilt.SetOutputMaximum(255)
    rescalFilt.SetOutputMinimum(0)
    itkimage = rescalFilt.Execute(sitk.Cast(sitktructedimage, sitk.sitkFloat32))
    return itkimage
def get_cube_from_img(img3d, center, block_size):
    # get roi(z,y,z) image and in order the out of img3d(z,y,x)range
    center_z = center[0]
    center_y = center[1]
    center_x = center[2]
    start_x = max(center_x - block_size / 2, 0)
    if start_x + block_size > img3d.shape[2]:
        start_x = img3d.shape[2] - block_size
    start_y = max(center_y - block_size / 2, 0)
    if start_y + block_size > img3d.shape[1]:
        start_y = img3d.shape[1] - block_size
    start_z = max(center_z - block_size / 2, 0)
    if start_z + block_size > img3d.shape[0]:
        start_z = img3d.shape[0] - block_size
    start_z = int(start_z)
    start_y = int(start_y)
    start_x = int(start_x)
    roi_img3d = img3d[start_z:start_z + block_size, start_y:start_y + block_size, start_x:start_x + block_size]
    return roi_img3d

In [86]:
img_file=r"D:\LUNA16\subset2/1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405.mhd"
itk_img = load_itkfilewithtrucation(img_file, 600, -1000)
img_array = sitk.GetArrayFromImage(itk_img)
    # x,y,z  Origin in world coordinates (mm)
origin = np.array(itk_img.GetOrigin())
    # spacing of voxels in world coor. (mm)
spacing = np.array(itk_img.GetSpacing())
    # go through all nodes
index = 0
classify_size=48

In [87]:
node_x = 89.32
node_y = 143.23
node_z = -427.1
# nodule center
center = np.array([node_x, node_y, node_z])
# nodule center in voxel space (still x,y,z ordering)  # clip prevents going out of bounds in Z
v_center = np.rint((center - origin) / spacing)
# convert x,y,z order v_center to z,y,z order v_center
v_center[0], v_center[1], v_center[2] = v_center[2], v_center[1], v_center[0]
# get cub size of classify_size
node_cube = get_cube_from_img(img_array, v_center, classify_size)
node_cube.astype(np.uint8)

array([[[ 13,  17,  17, ...,  11,  13,  12],
        [ 15,  16,  16, ...,  13,  14,  11],
        [  9,  11,  11, ...,  11,   9,  10],
        ...,
        [229, 222, 211, ...,  20,  18,  16],
        [222, 214, 206, ...,  19,  21,  29],
        [216, 208, 194, ...,  19,  37,  70]],

       [[ 10,  15,  21, ...,  11,  11,  10],
        [ 14,  20,  23, ...,  16,  14,  11],
        [ 15,  18,  19, ...,  15,  10,  10],
        ...,
        [201, 179, 147, ...,  19,   9,   7],
        [182, 159, 133, ...,  22,  20,  19],
        [166, 145, 120, ...,  27,  41,  56]],

       [[  7,   6,  13, ...,   7,   8,  11],
        [  7,   7,  10, ...,  10,  10,  11],
        [  7,   6,   9, ...,  10,  10,  12],
        ...,
        [135, 110,  77, ...,  14,   7,   7],
        [110,  85,  60, ...,  14,  11,  11],
        [ 88,  71,  57, ...,  14,  25,  38]],

       ...,

       [[ 18,  24,  28, ...,  15,  13,  18],
        [ 18,  26,  31, ...,  21,  19,  22],
        [ 15,  23,  28, ...,  25,  18,  18

In [88]:
filepath=img_file[10:13]
print(filepath)
np.save(filepath + ".npy", node_cube)

sub


In [89]:
i=r"C:\Users\dillu\MAINPROJECT/sub.npy"
print(i)
fig1=[]
img_array = np.load(r"C:\Users\dillu\MAINPROJECT/sub.npy")
img_array=img_array.reshape(192,192,3)
fig1.append(img_array)

C:\Users\dillu\MAINPROJECT/sub.npy


In [90]:
print(fig1)

[array([[[ 13.865625,  17.53125 ,  17.2125  ],
        [ 14.821875,  15.3     ,  19.921875],
        [ 24.225   ,  20.240625,  11.79375 ],
        ...,
        [  7.809375,  10.040625,  14.184375],
        [ 26.615625,  48.76875 ,  59.925   ],
        [ 59.128124,  50.68125 ,  32.034374]],

       [[ 21.0375  ,  27.253124,  27.09375 ],
        [ 14.025   ,   6.05625 ,  11.475   ],
        [ 19.284374,  24.065624,  23.746876],
        ...,
        [ 17.371876,  20.71875 ,  22.153126],
        [ 21.196875,  21.35625 ,  22.153126],
        [ 22.3125  ,  15.9375  ,  10.8375  ]],

       [[ 27.571875,  15.61875 ,  11.475   ],
        [ 14.98125 ,  30.440624,  47.334373],
        [ 43.190624,  31.715626,  25.659374],
        ...,
        [  6.05625 ,   3.50625 ,   9.721875],
        [ 16.25625 ,  16.25625 ,  11.315625],
        [  6.375   ,   3.665625,   4.78125 ]],

       ...,

       [[ 16.734375,  19.921875,  21.0375  ],
        [ 33.946877,  63.590626,  87.3375  ],
        [ 93.39375 , 

In [91]:
X_test = np.array(fig1)
X_test = X_test/255.0
X_test = np.asarray(X_test).astype(np.float32)

In [92]:
from keras.models import load_model
model=load_model("VGG16new.h5")

In [93]:
y_pred1 = np.argmax(model.predict(X_test), axis=1)

In [94]:
print(y_pred1)

[0]


In [ ]:
1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492
-100.70966
68.191806
